In [1]:
from IPython.display import YouTubeVideo
import cv2
import numpy as np
import os
import math
from matplotlib import pyplot as plt
from IPython.display import clear_output # Extra
#multiple cascades: https://github.com/Itseez/opencv/tree/master/data/haarcascades
newpath = 'C:/Users/ViksIsBck/Anaconda3/pkgs/opencv-3.4.4-py36hb76ac4c_1203/Library/etc/haarcascades/'
#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_cascade = cv2.CascadeClassifier(newpath + 'haarcascade_frontalface_default.xml')
#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml
eye_cascade = cv2.CascadeClassifier(newpath + 'haarcascade_eye.xml')

In [2]:
def cut_faces(image, faces_coord):
    faces = []
      
    for (x, y, w, h) in faces_coord:
        w_rm = int(0.2 * w / 2)
        faces.append(image[y: y + h, x + w_rm: x + w - w_rm])
         
    return faces

In [3]:
def normalize_intensity(images):
    images_norm = []
    for image in images:
        is_color = len(image.shape) == 3 
        if is_color:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images_norm.append(cv2.equalizeHist(image))
    return images_norm

In [4]:
def resize(images, size=(50, 50)):
    images_norm = []
    for image in images:
        if image.shape < size:
            image_norm = cv2.resize(image, size, 
                                    interpolation = cv2.INTER_AREA)
        else:
            image_norm = cv2.resize(image, size, 
                                    interpolation = cv2.INTER_CUBIC)
        images_norm.append(image_norm)

    return images_norm 

In [5]:
def normalize_faces(frame, faces_coord):
    faces = cut_faces(frame, faces_coord)
    faces = normalize_intensity(faces)
    faces = resize(faces)
    return faces

def draw_rectangle(image, coords):
    for (x, y, w, h) in coords:
        w_rm = int(0.2 * w / 2) 
        cv2.rectangle(image, (x + w_rm, y), (x + w - w_rm, y + h), 
                              (150, 150, 0), 8)

In [9]:
folder = "Desktop/People/" + input('Person: ').lower() # input name

Person: Vivek


In [10]:
import os
os.mkdir(folder)
counter = 1
timer = 0

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'Desktop/People/vivek'

In [11]:
counter=0
cap = cv2.VideoCapture(0)
while counter < 21 : # take 20 pictures
        ret, frame = cap.read()
        faces_coord = face_cascade.detectMultiScale(frame, 1.3, 5)

        
        
        
        if len(faces_coord) and timer % 700 == 50: 
            faces = normalize_faces(frame, faces_coord) 
            cv2.imwrite(folder + '/' + str(counter) + '.jpg', faces[0])
            clear_output(wait = True) #saved face in notebook
            counter += 1
            
        draw_rectangle(frame, faces_coord) #rectangle around face
        
        cv2.imshow('Mirror Mirror On The Wall',frame)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
        timer += 50

cap.release()        
cv2.destroyAllWindows()

In [13]:
def collect_dataset():
    images = []
    labels = []
    labels_dic = {}
    people = [person for person in os.listdir('C:/Users/ViksIsBck/Desktop/People/')]
    for i, person in enumerate(people):
        labels_dic[i] = person
        for image in os.listdir('C:/Users/ViksIsBck/Desktop/People/' + person):
            images.append(cv2.imread("C:/Users/ViksIsBck/Desktop/People/" + person + '/' + image, 
                                     0))
            labels.append(i)
    return (images, np.array(labels), labels_dic)

In [14]:
images, labels, labels_dic = collect_dataset()

rec_lbph = cv2.face.LBPHFaceRecognizer_create()
rec_lbph.train(images, labels)

print ("Models Trained Succesfully")

Models Trained Succesfully


In [ ]:
cap = cv2.VideoCapture(0)
while True: 
        ret, frame = cap.read()
        faces_coord = face_cascade.detectMultiScale(frame, 1.3, 5)

        
        
        
        if len(faces_coord):
            faces = normalize_faces(frame, faces_coord) 
            for i, face in enumerate(faces): # for each detected face
                collector = cv2.face.StandardCollector_create()
                rec_lbph.predict_collect(face, collector)
                conf = collector.getMinDist()
                pred = collector.getMinLabel()
                threshold = 140
                print ("Prediction: " + labels_dic[pred].capitalize() + "\nConfidence: " + str(round(conf)))
                cv2.putText(frame, labels_dic[pred].capitalize(),
                        (faces_coord[i][0], faces_coord[i][1] - 10),
                        cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
            
            clear_output(wait = True)
            draw_rectangle(frame, faces_coord)
        
        cv2.putText(frame, "ESC to exit", (5, frame.shape[0] - 5),
                    cv2.FONT_HERSHEY_PLAIN, 1.3, (66, 53, 243), 2, cv2.LINE_AA)
        cv2.imshow("PyData Tutorial", frame) # live feed in external
        if cv2.waitKey(40) & 0xFF == 27:
            cv2.destroyAllWindows()
            break

Prediction: Somar
Confidence: 135
